In [1]:
%load_ext autoreload
%autoreload 2

import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Anchor.anchor import Anchor, Tasktype

np.random.seed(1)

In [2]:
# dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/
# Link: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
def load_polarity(path='/home/kevin/iml-ws21-projects-risingnumpygods/datasets/rt-polaritydata/'):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(l)
    return data, labels

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
data, labels = load_polarity()
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)
val_vectors = vectorizer.transform(val)

In [5]:
c = sklearn.linear_model.LogisticRegression()

c.fit(train_vectors, train_labels)

preds = c.predict(val_vectors)

print('Val accuracy', sklearn.metrics.accuracy_score(val_labels, preds))

def predict_lr(texts):
    print(vectorizer.transform(texts))
    return c.predict(vectorizer.transform(texts))

Val accuracy 0.7544910179640718


/home/kevin/miniconda3/envs/iML/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
text_to_be_explained = "This is a good book."

explainer = explainer = Anchor(Tasktype.TEXT)

task_paras = {"nlp_object": spacy.load("en_core_web_sm")}
method_paras = {"beam_size": 1, "desired_confidence": 0.8}
explainer.explain_instance(
    text_to_be_explained,
    predict_lr,
    method="beam",
    method_specific=method_paras,
    task_paras = {"nlp_object": spacy.load("en_core_web_sm")}
    num_coverage_samples=1
)

TypeError: __init__() missing 1 required positional argument: 'nlp_object'